#Download data

In [1]:
# Download the training data
!gdown "https://drive.google.com/uc?export=download&id=199EtucyfnhZOIHq_jfGjODTZm3Scz5yC"

Downloading...
From: https://drive.google.com/uc?export=download&id=199EtucyfnhZOIHq_jfGjODTZm3Scz5yC
To: /content/quora_questions_train.csv
100% 124M/124M [00:00<00:00, 126MB/s]


In [2]:
# Download the test data
!gdown "https://drive.google.com/uc?export=download&id=1hL9DZvEY-HCOh3FXVtUIuS1CyM8zhRZn"

Downloading...
From: https://drive.google.com/uc?export=download&id=1hL9DZvEY-HCOh3FXVtUIuS1CyM8zhRZn
To: /content/quora_questions_test.csv
100% 35.0M/35.0M [00:00<00:00, 73.9MB/s]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from tqdm import tqdm
import numpy as np
SEED = 23

In [5]:
class BagofwordsDataset(torch.utils.data.Dataset):
    def __init__(self, data, vectorizer):
        self.df = data
        self.sequences = vectorizer.transform(self.df.question_text.tolist()) # matrix of word counts for each sample
        self.labels = self.df.target.tolist() # list of labels
        self.token2idx = vectorizer.vocabulary_ # dictionary converting words to their counts
        self.idx2token = {idx: token for token, idx in self.token2idx.items()} # same dictionary backwards
    def __getitem__(self, i):
        # return the ith sample's list of word counts and label
        return self.sequences[i, :].toarray(), self.labels[i]

    def __len__(self):
        return self.sequences.shape[0]

In [6]:
class BagOfWordsNetwork(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, vectorizer):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()
        self.vectorizer = vectorizer

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [12]:
def bagofwords_train(train_dataset, val_dataset, model, hyperparameters, n_eval):
    """
    Trains and evaluates a model.
    Args:
        train_dataset:   PyTorch dataset containing training data.
        val_dataset:     PyTorch dataset containing validation data.
        model:           PyTorch model to be trained.
        hyperparameters: Dictionary containing hyperparameters.
        n_eval:          Interval at which we evaluate our model.
    """

    # Get keyword arguments
    batch_size, epochs = hyperparameters["batch_size"], hyperparameters["epochs"]

    # Initialize dataloaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )

    # Note: batch_size = len(val_dataset), so that's the whole validation set
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=len(val_dataset), shuffle=True
    )

    # Initalize optimizer (for gradient descent) and loss function
    optimizer = optim.Adam(model.parameters())
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1} of {epochs}")

        # Loop over each batch in the dataset
        for batch, (X, y) in tqdm(enumerate(train_loader)):
            # Predictions and loss
            X = X.type(torch.float)
            y = y.type(torch.float)

            pred = model(X)
            pred = np.squeeze(pred)
            loss = loss_fn(pred, y)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Periodically evaluate our model + log to Tensorboard
            if batch % n_eval == 0:
                # CHANGE PATH AS NECESSARY
                joshua_path = "/content/drive/MyDrive/ACM AI Stuff/BagOfWordsSaves/"
                #arjun_path = ""
                name = "BagOfWords" + str(batch) + ".pt"
                torch.save(model, joshua_path + name)
                #torch.save(model, arjun_path + name)

                # Compute training loss and accuracy.
                accuracy = compute_accuracy(pred, y)
                print("batch loss: ", loss)
                print("batch accuracy: ", accuracy)

                # Compute validation loss and accuracy.
                val_loss, val_accuracy, f1 = bagofwords_evaluate(val_loader, model, loss_fn)
                print("validation loss: ", val_loss)
                print("validation accuracy: ", val_accuracy)
                print("f1 score: ", f1)



def compute_accuracy(outputs, labels):
    n_correct = (torch.round(outputs) == labels).sum().item()
    n_total = len(outputs)
    return n_correct / n_total


def bagofwords_evaluate(val_loader, model, loss_fn):
    with torch.no_grad():
        # There should only be one batch (the entire validation set)
        for (X, y) in val_loader:
            X = X.type(torch.float)
            y = y.type(torch.float)

            pred = model(X)
            pred = np.squeeze(pred)
            loss = loss_fn(pred, y)
            f1 = f1_score(torch.round(pred), y, average='macro')
            accuracy = compute_accuracy(pred, y)
            return loss, accuracy, f1

#Training

In [8]:
EPOCHS = 1
BATCH_SIZE = 32
N_EVAL = 100
HIDDEN_DIM = 64
hyperparameters = {"batch_size": BATCH_SIZE, "epochs": EPOCHS}

In [9]:
%cd /
%cd content
data_path = 'quora_questions_train.csv'
data_pd = pd.read_csv(data_path)
data, val = train_test_split(data_pd, test_size = 0.05, stratify = data_pd['target'], shuffle = True, random_state = SEED)
vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
fit = vectorizer.fit(data.question_text.tolist())
train_dataset = BagofwordsDataset(data,fit)
val_dataset = BagofwordsDataset(val,fit)

/
/content


In [10]:
model = BagOfWordsNetwork(train_dataset.sequences.shape[1], HIDDEN_DIM, fit)

In [ ]:
bagofwords_train(
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        model=model,
        hyperparameters=hyperparameters,
        n_eval=N_EVAL,
    )

#Evaluation

In [14]:
joshua_saves_path = "/content/drive/MyDrive/ACM AI Stuff/BagOfWordsSaves/"
model_name = joshua_saves_path + "BagOfWords38700.pt"

trained_model = torch.load(model_name)
trained_model.eval()

BagOfWordsNetwork(
  (fc1): Linear(in_features=110, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [15]:
%cd /
%cd content
test_path = 'quora_questions_test.csv'
test_pd = pd.read_csv(test_path)

/
/content


In [16]:
test_pd.head()

,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [17]:
test_size = test_pd["qid"].size
test_size

375806

In [18]:
test_pd.loc[0, "question_text"]

'Why do so many women become so rude and arrogant when they get just a little bit of wealth and power?'

In [23]:
test_sequences = trained_model.vectorizer.transform(test_pd.question_text.tolist())

In [26]:
test_sequences

<375806x110 sparse matrix of type '<class 'numpy.int64'>'
	with 421977 stored elements in Compressed Sparse Row format>

In [29]:
from tqdm.notebook import tqdm

guesses = []
for i in tqdm(range(test_size)):
  input = torch.tensor(test_sequences[i, :].toarray()).type(torch.float)
  pred = trained_model(input)
  pred = torch.round(torch.squeeze(pred)).item()
  guesses.append(pred)

  0%|          | 0/375806 [00:00<?, ?it/s]

In [31]:
guesses[0:5]

[0.0, 0.0, 0.0, 0.0, 0.0]